# HEART FAILURE ANALYSIS - PYTHON

Hi everyone, I'm Tasdeed and this is my first submission on Kaggle and this is my first real hands Data Science project as well as first time using Python after an online course.
I have never done machine learning before, even more so in Python and with help from other notebooks and online resources I managed to implement some methods, so the accuracies of the different algorithms I used could most definitely be improved once I learn the different sorts of parameters these models take!
I would love for some feedback as well from more experienced people on certain things I can implement or further resources!
If this notebook is good enough I would appreciate it if the reader upvotes this!

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Relevant Libraries for analysis:

In [ ]:
#Data Wrangling:
import numpy as np
import pandas as pd

#Data Visualization:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Workflow Stage 1: Data Importation

In [ ]:
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head(10)

In [ ]:
df.shape

## Meaning of Variables:

Age - Self Explanatory

Anaemia - Is 1 or 0 with 1 being the patient does have this condition.
Anemia is a condition in which you lack enough healthy red blood cells to carry adequate oxygen to your body's tissues.

Creatinine Phosphokinase - Level of CPK enzyme in the blood

Diabetes - Is a 1 or 0 - whether the patient suffers from diabetes or not - similar to anaemia

Ejection Fraction - Is a percentage (numerical between 0 to 100)
Ejection fraction is a measurement of the percentage of blood leaving your heart each time it contracts.

High Blood Pressure - Is a 1 or 0 - whether patient suffers from high blood pressure

Platelets - no. of platelets in the blood

Serum Creatine - Level of Creatine produced from the kidneys in the blood

Serum Sodium - Level of Sodium in the blood

Sex - Self Explanatory - assuming 1 is male and 0 is female

Smoking - Self Explanatory - assuming 1 is smokes and 2 is doesn't smoke

Time - Follow up days

Death Event - whether patient died during follow up period

## About the Dataset

Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

## What sort of datatypes do we have?

In [ ]:
df.info()

We can see all features are numerical variables

## Assessing for null-values:

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Searching for missing values:')
sns.heatmap(data=df.isnull(),cmap = 'coolwarm', cbar = False)

We have completely filled data - that's not to say however that we don't have any outliers!

## What types of features do we have?

Continuous: Age, CPK, Ejection Fraction, Platelets, Serum Creatine, Serum Sodium and Time

Categorical: Anaemia, Diabetes, High Blood Pressue, Sex, Smoking, Death Event

We do not have any variables with mixed data types.

## Looking for features which may contain errors/typos

In [ ]:
df.describe()

### Conclusions:

1. We seem to have outliers for CPK - since the max value is way higher than expected from the 75% quartile. With the same reasoning, this is the same for Platelets (+ fact that min is 25100) and Serum Creatinine and ejection fraction and possibly time. These need to be corrected
2. The remaining variables: Continuous (Age, Serum Sodium, Time) and Categoricals (remaining 5 variables) are fine - in total we've accounted for all 12 variables - excluding Death Event.

Aside from this:
1. 96 of the 299 (32%) patients in the sample passed away

## Assessing the distribution of the features:

## Continuous Features:

In [ ]:
plt.style.use('ggplot')
fig, axis  = plt.subplots(nrows = 3, ncols = 2, figsize = (15,9))

ax0, ax1, ax2, ax3, ax4, ax5 = axis.flatten()

ax0.hist(df['age'])
ax0.set_xlabel('Age')

ax1.hist(df['creatinine_phosphokinase'])
ax1.set_xlabel('CPK Enzyme')

ax2.hist(df['platelets'], bins = 15)
ax2.set_xlabel('Platelets')

ax3.hist(df['serum_creatinine'])
ax3.set_xlabel('Serum Creatinine')

ax4.hist(df['serum_sodium'], bins = 15)
ax4.set_xlabel('Serum Sodium')

ax5.hist(df['ejection_fraction'])
ax5.set_xlabel('Ejection Fraction')

plt.tight_layout()

### Conclusions:

- The majority of patients were aged between 50-65 - with around a third of patients above 65 
- Our remarks from above regarding the possibility of outliers have been confirmed as shown by the plots for the CPK enzyme, platelet counts, creatinine, sodium and ejection fraction
- Most patients CPK in fact were below 1000
- Serum Creatinine was less than 2 (>75%)
- Platelets varied between 150K-400K
- Sodium varied between 132-142
- Ejection Fraction varied significantly amongst patients with some as low as 15% as high as 55% excluding outliers

## Now looking at distributions of the categorical variables:

In [ ]:
fig, ax  = plt.subplots(nrows = 3, ncols = 2, figsize = (12,6))
plt.tight_layout()
sns.countplot(df['anaemia'], ax=ax[0,0])
sns.countplot(df['diabetes'], ax=ax[0,1])
sns.countplot(df['high_blood_pressure'], ax=ax[1,0])
sns.countplot(df['sex'], ax=ax[1,1])
sns.countplot(df['smoking'], ax=ax[2,0])
fig.delaxes(ax[2,1])

In [ ]:
x1 = (len(df[df['anaemia'] == 1]))/len(df['anaemia'])
x2 = (len(df[df['diabetes'] == 1]))/len(df['diabetes'])
x3 = (len(df[df['high_blood_pressure'] == 1]))/len(df['high_blood_pressure'])
x4 = (len(df[df['sex'] == 1]))/len(df['sex'])
x5 = (len(df[df['smoking'] == 1]))/len(df['smoking'])

data = {'Anaemia': x1, 'Diabetes': x2, 'High Blood Pressure': x3, 'Sex': x4, 
       'Smoking': x5}
categ_zeros = pd.DataFrame(data, index=[1])
categ_zeros

## Conclusion:

- The majority of patients in this sample (65%) are male by assumption that 1 = Male
- Around a third of patients suffered from either High Blood Pressure, Smoking or both
- Around 40% of the sample either had anaemia or diabeters or both too.

### Assumptions based on data analysis

We arrive at following assumptions based on data analysis done so far. We may validate these assumptions further before taking appropriate actions.

**Completing.**

- No variables require completing

**Correcting.**

1. We may want to correct the above mentioned variables in the event we find that there is some correlation with survival


**Creating.**

1. We may want create bands for age given that patients become more vulnerable as the get older.
2. Given how broad the time feature is in terms of values - we may want to also create a band for this if there is indeed a relationships with survival
3. We may want to create a new column denoting whether the patient had any sort of underlying condition i.e. anaemia, diabetes or high blood pressure

**Classifying.**

Additional assumptions:

1. Women may have higher chances of survival than men
2. Those who had less time in follow up days had higher chances of survival
3. Those who were younger than a certain age had higher chances of survival

## Analyzing relationships between survival and categorical variables via grouping:

In [ ]:
df[['sex', 'DEATH_EVENT']].groupby(['sex'], as_index = False).mean()

In [ ]:
df[['smoking', 'DEATH_EVENT']].groupby(['smoking'], as_index = False).mean()

In [ ]:
df[['anaemia', 'DEATH_EVENT']].groupby(['anaemia'], as_index = False).mean()

In [ ]:
df[['diabetes', 'DEATH_EVENT']].groupby(['diabetes'], as_index = False).mean()

In [ ]:
df[['high_blood_pressure', 'DEATH_EVENT']].groupby(['high_blood_pressure'], 
                                                   as_index = False).mean()

## Conclusions:

What we notice here is that there does not seem to be any correlation between  whether the patients had any underlying health conditions, their sex or whether they smoked against whether they died or not.
In conclusion we can say assumptions 1 and 4 made above are false.

Before we can discard these variables - it is best to see if we see if we can create a new feature out of these before dropping them. For example - we could create a new categorical feature denoting whether the patient has an underlying condition or not (diabetes, anaemia, high blood pressure). For sex and smoking - it seems so that we should drop them but before we do this, we do so we should check for underlying relationships with numerical variables.

## Analyzing relationships between survival and numerical variables:

Firstly a heatmap will come in very useful.

We immediately notice that there appears to be correlation between death with either age, no. of follow up days, ejection fraction and serum sodium.

In [ ]:
plt.figure(figsize = (12,6))
sns.heatmap(df[['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium', 'time', 'DEATH_EVENT']].corr(), cmap = 'coolwarm', annot = True)

In [ ]:
g = sns.FacetGrid(df, col = 'DEATH_EVENT')
g.map(plt.hist, 'age')

In [ ]:
g = sns.FacetGrid(df, col = 'DEATH_EVENT')
g.map(plt.hist, 'time')

In [ ]:
g = sns.FacetGrid(df, col = 'DEATH_EVENT')
g.map(plt.hist, 'ejection_fraction')

In [ ]:
g = sns.FacetGrid(df, col = 'DEATH_EVENT')
g.map(plt.hist, 'serum_creatinine')

## Conclusions: 

1. Those over 70 had higher chances of dying, whilst many under 50 had higher chances of surival suggesting the need to band ages. So assumption #3 is true!
2. Those with a larger amount of follow up days tended to survive whilst those with less (<50) ended up with a higher mortality - for model building purposes we would like to band this too. This was in fact the opposite our assumption #2.
3. With regards to the ejection fraction, what we can see it that with high percentages had lower deaths. Those with lower percentages (<30%) had more deaths - again suggesting that we should create some bands.
4. Patients with higher creatinine levels suffered more deaths whist the vast majority (75%) who had percentages less than 1.4 has greater survival rates.
5. Platelets and CPK enzymes do not seem to have an effect with survival as well as with other variables - it makes sense to drop these

We can conclude that we have 4 numerical features which are all useful for our model building and that these features need to be sorted out in terms of outliers should they contain any. We would also like to create bands on the four of these.

# Analyzing relationships between categorical and numerical variables:

In [ ]:
g = sns.FacetGrid(df, col = 'DEATH_EVENT', row = 'sex')
g.map(plt.hist, 'age')

## Conclusion

1. Looking at the plot above - for smoking against the age - we either don't see much relationship or we reach similar conclusions from above even when we split into whether a patient smokes or not - the same can be said for smoking against the other 3 relevant numerical variables. This means assumption #1 is false.
2. The same can be said for sex - overall this strengthens our case for dropping the two variables.

# Data Wrangling:

1. From the above - we have overall said that our only useful features at the moment are: age, time, serum creatinine and the ejection fraction. 
2. At the same time we have said that we can create a new column denoting whether a patient has any underlying condition whether it be anaemia, high blood pressure or diabetes.
3. The remaining features can be dropped

So all we need to do is create the new feature - assess how useful it is and decide whether we need to drop this, we need to fix for outliers for our useful features and drop the relevant features we no longer require before we can begin the prediction stage. We also need to band relevant features too.

## Feature Dropping:

In [ ]:
print("Before", df.shape)
df = df.drop(['platelets', 'creatinine_phosphokinase', 'serum_sodium', 'sex', 'smoking'], axis = 1)
print("After", df.shape)

## Creating new underlying condion column (UnderCon):

In [ ]:
df['UnderCon'] = 0
df.loc[((df['anaemia'] == 1) | (df['diabetes'] == 1) | df['high_blood_pressure'] == 1), 'UnderCon'] = 1
df.drop(['anaemia', 'diabetes', 'high_blood_pressure'], axis = 1, inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
df[['DEATH_EVENT', 'UnderCon']].groupby('UnderCon', as_index = False).mean()

## Conclusion:

There doesn't appear to be any correlation here, so we can just go ahead and drop this leaving us with our 4 main features to use in our model training - now we just need to deal with our outliers.

In [ ]:
df = df.drop('UnderCon', axis = 1)

In [ ]:
df.head()

## Outlier Removal

As mentioned at the beginning of the analysis - considering the remaining features we have left - the ones that contain possible outliers are ejection fraction, time and serum creatinine

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (12,6))
plt.tight_layout()

ax0,ax1,ax2,ax3 = ax.flatten()

ax0.boxplot(df['serum_creatinine'])
ax0.set_title('Serum Creatinine')

ax1.boxplot(df['ejection_fraction'])
ax1.set_title('Ejection Fraction')

ax2.boxplot(df['age'])
ax2.set_title('Age')

ax3.boxplot(df['time'])
ax3.set_title('time')

## Conclusion:

Before we even go about removing outliers we need to understand typical values for these features - one just can't remove outliers! 

For example - the ejection fraction of a healthy person is typically between 50 -70% so a patient having a percentage between 70-80% which was once thought as an outlier isn't necessarily even though the boxplot suggests this. 

For Serum Creatinine - it seems there are very large values distant from the rest of the datapoints suggesting the existence of outliers (in this case values larger than say 3.5.) - to correct this - we'll replace values larger than 4 with the median of the rest of the values. 
NOTE - whilst the outliers occur at values higher than around 2 rather than 3.5 as shown by the boxplot and using the fact that the internet says a normal human being has level at around 1.2 - I could have removed all those outliers after 2 - however I took into account that there may be some patients with underlying heart conditions not taken into account by the data.

We have no outliers for Age and Time.

Hence, we only need to fix values for serum creatinine levels.

In [ ]:
serum_cmode = df['serum_creatinine'].median()
df.loc[(df['serum_creatinine'] > 4), 'serum_creatinine'] = serum_cmode
df.head(10)

## Banding features:

Age

- As we can see banding was effective - we see clear correlation!

In [ ]:
df['AgeBand'] = pd.cut(df['age'],5)
df[['AgeBand', 'DEATH_EVENT']].groupby('AgeBand', as_index = False).mean()

In [ ]:
#Mapping:

df.loc[df['age'] <= 51, 'age'] = 0
df.loc[(df['age'] > 51) & (df['age'] <= 62), 'age'] = 1
df.loc[(df['age'] > 62) & (df['age'] <= 73), 'age'] = 2
df.loc[(df['age'] > 73) & (df['age'] <= 84), 'age'] = 3
df.loc[(df['age'] > 84) & (df['age'] <= 95), 'age'] = 4

df.drop('AgeBand', axis = 1, inplace = True)

Time
- Similar Conclusion.

In [ ]:
df['TimeBand'] = pd.cut(df['time'],5)
df[['TimeBand', 'DEATH_EVENT']].groupby('TimeBand', as_index = False).mean()

In [ ]:
#Mapping:

df.loc[df['time'] <= 60.2, 'time'] = 0
df.loc[(df['time'] > 60.2) & (df['time'] <= 116.4), 'time'] = 1
df.loc[(df['time'] > 116.4) & (df['time'] <= 172.6), 'time'] = 2
df.loc[(df['time'] > 172.6) & (df['time'] <= 228.8), 'time'] = 3
df.loc[(df['time'] > 228.8) & (df['time'] <= 285), 'time'] = 4

df.drop('TimeBand', axis = 1, inplace = True)

Serum Creatinine
- Similar Conclusion.

In [ ]:
df['SCBand'] = pd.cut(df['serum_creatinine'],5)
df[['SCBand', 'DEATH_EVENT']].groupby('SCBand', as_index = False).mean()

In [ ]:
#Mapping:

df.loc[df['serum_creatinine'] <= 1.2, 'serum_creatinine'] = 0
df.loc[(df['serum_creatinine'] > 1.2) & (df['serum_creatinine'] <= 1.9), 'serum_creatinine'] = 1
df.loc[(df['serum_creatinine'] > 1.9) & (df['serum_creatinine'] <= 2.6), 'serum_creatinine'] = 2
df.loc[(df['serum_creatinine'] > 2.6) & (df['serum_creatinine'] <= 3.3), 'serum_creatinine'] = 3
df.loc[(df['serum_creatinine'] > 3.3) & (df['serum_creatinine'] <= 4), 'serum_creatinine'] = 4

df.drop('SCBand', axis = 1, inplace = True)

Ejection Fraction
- Similar Conclusion.

In [ ]:
df['EJBand'] = pd.cut(df['ejection_fraction'],5)
df[['EJBand', 'DEATH_EVENT']].groupby('EJBand', as_index = False).mean()

In [ ]:
#Mapping:

df.loc[df['ejection_fraction'] <= 27.2, 'ejection_fraction'] = 0
df.loc[(df['ejection_fraction'] > 27.2) & (df['ejection_fraction'] <= 40.4), 'ejection_fraction'] = 1
df.loc[(df['ejection_fraction'] > 40.4) & (df['ejection_fraction'] <= 53.6), 'ejection_fraction'] = 2
df.loc[(df['ejection_fraction'] > 53.6) & (df['ejection_fraction'] <= 66.8), 'ejection_fraction'] = 3
df.loc[(df['ejection_fraction'] > 66.8) & (df['ejection_fraction'] <= 80), 'ejection_fraction'] = 4

df.drop('EJBand', axis = 1, inplace = True)

### Converting age and serum creatinine to integers:

In [ ]:
#Converting age and serum creatinine to integers:
df['age'] = df['age'].astype(int)
df['serum_creatinine'] = df['serum_creatinine'].astype(int)
df.head()

# Modelling and Prediction:

This is a classification and a regression problem since we are predicting an output based off of other independent features. Furthermore - this is a supervised problem as we will have a training dataset to train our model against.

Possible models:
- Logistic Regression
- KNN or k-Nearest Neighbors
- Support Vector Machines
- Naive Bayes classifier
- Decision Tree
- Random Forrest
- Perceptron
- Artificial neural network
- RVM or Relevance Vector Machine

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train, test = train_test_split(df, test_size = 0.2, random_state = 42)
print(train.head())
print('_'*40)
print(test.head())

In [ ]:
X_train = train.drop('DEATH_EVENT', axis = 1)
Y_train = train['DEATH_EVENT']
X_test = test.drop('DEATH_EVENT', axis=1).copy()
Y_test = test['DEATH_EVENT']

X_train.shape,Y_train.shape,X_test.shape, Y_test.shape

## Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)

acc_log_train = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_log_train,'%')
print('-'*25)
print('Test Score:',acc_log_test,'%')

## Support Vector Machines

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)

acc_svc_train = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_svc_train,'%')
print('-'*25)
print('Test Score:',acc_svc_test,'%')


## K Nearest-Neighhbours (KNN)

In [ ]:
error_rate = []
for i in range(1,40):
 knn = KNeighborsClassifier(n_neighbors=i)
 knn.fit(X_train,Y_train)
 pred_i = knn.predict(X_test)
 error_rate.append(np.mean(pred_i != Y_test))

plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', 
         marker='o',markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')
print("Minimum error:-",min(error_rate),"at K =",error_rate.index(min(error_rate)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = error_rate.index(min(error_rate)))
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)

acc_knn_train = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_knn_train,'%')
print('-'*25)
print('Test Score:',acc_knn_test,'%')

## Gaussian Naive-Bayes

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)

acc_gaussian_train = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_gaussian_train,'%')
print('-'*25)
print('Test Score:',acc_gaussian_test,'%')

## Perceptron

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)

acc_perceptron_train = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_perceptron_train,'%')
print('-'*25)
print('Test Score:',acc_perceptron_test,'%')

## Linear SVC

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)

acc_linear_svc_train = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_linear_svc_train,'%')
print('-'*25)
print('Test Score:',acc_linear_svc_test,'%')

## Stochastic Gradient Descent

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)

acc_sgd_train = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_sgd_train,'%')
print('-'*25)
print('Test Score:',acc_sgd_test,'%')

## Decision Tree

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)

acc_decision_tree_train = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_decision_tree_train,'%')
print('-'*25)
print('Test Score:',acc_decision_tree_test,'%')

## Random Forest

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

acc_random_forest_train = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest_test = round(accuracy_score(Y_pred, Y_test) * 100,2)

print('Training Score:',acc_random_forest_train,'%')
print('-'*25)
print('Test Score:',acc_random_forest_test,'%')

# Model Evaluation:

## Score under the training set:

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc_train, acc_knn_train, acc_log_train, 
              acc_random_forest_train, acc_gaussian_train, acc_perceptron_train, 
              acc_sgd_train, acc_linear_svc_train, acc_decision_tree_train]})
models.sort_values(by='Score', ascending=False)

## Score under the test set:

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc_test, acc_knn_test, acc_log_test, 
              acc_random_forest_test, acc_gaussian_test, acc_perceptron_test, 
              acc_sgd_test, acc_linear_svc_test, acc_decision_tree_test]})
models.sort_values(by='Score', ascending=False)

# Conclusions:

- It would seem that the Linear SVC model produces our best results of 78.3% accuracy on the test set with the best model under the training set being the Random Forest with 94% accuracy
- There is clear room for improvement if we can adjust hyperparameters using cross-validation (currently unsure of how to do this with current knowledge)
    (Note to self: come back to this later and attempt to improve scores!)
    
- This concludes this notebook, feel free to give some feedback on improvements or any available to resources to learn from!
- Up-vote if you think this is good!